# Remote Sensing-Based Monitoring of Active Restoration in the Atlantic Rainforest: Hands-on Notebook

This notebook serves as an implementation guide for identifying active restoration in the Atlantic Rainforest using remote sensing imagery and Machine Learning. Developed by Felipe Begliomini for his MRes dissertation in the AI4ER program at the University of Cambridge, it provides users with step-by-step instructions to effectively utilize calibrated models and gain valuable insights into active restoration efforts within the Atlantic Rainforest. For further information about the methodology, please refer to the accompanying report available at the GitHub repository.

The notebook is  designed for usage within the Google Colaboratory environment once it offers free access to GPU resources. To access the notebook, visit the [Colab Website](https://colab.research.google.com), click on "File," then select "Open notebook," followed by "GitHub." Copy and paste the following link: https://github.com/fnincao/reforestation-CNN.git. Once opened, navigate to "Edit," then select "Notebook settings," and switch the hardware accelerator to GPU. If you encounter any difficulties activating the GPU, you can refer to this [link](https://medium.com/dataman-in-ai/start-using-google-colab-free-gpu-7968acb7ef92) for further assistance.

After following these instructions, run the cells and proceed as instructed.

## Clone GitRepo

In [ ]:
!git clone https://github.com/fnincao/reforestation-CNN.git 

## Download Pre-trained models

To apply the Machine Learning models, it's necessary to download the training checkpoints from the [Zenodo repository](https://doi.org/10.5281/zenodo.8111658). Please run the following code cells to download the files directly to the `/checkpoints folder`:

In [ ]:
!wget -O reforestation-CNN/checkpoints/fusion.pth.tar "https://zenodo.org/record/8111751/files/fusion.pth.tar?download=1"

In [ ]:
!wget -O reforestation-CNN/checkpoints/fusion_ne.pth.tar "https://zenodo.org/record/8111751/files/fusion_ne.pth.tar?download=1"

In [ ]:
!wget -O reforestation-CNN/checkpoints/ndvi.pth.tar "https://zenodo.org/record/8111751/files/ndvi.pth.tar?download=1"

In [ ]:
!wget -O reforestation-CNN/checkpoints/ndvi_ne.pth.tar "https://zenodo.org/record/8111751/files/ndvi_ne.pth.tar?download=1"

In [ ]:
!wget -O reforestation-CNN/checkpoints/planet.pth.tar "https://zenodo.org/record/8111751/files/planet.pth.tar?download=1"

In [ ]:
!wget -O reforestation-CNN/checkpoints/planet_ne.pth.tar  "https://zenodo.org/record/8111751/files/planet_ne.pth.tar?download=1"

In [ ]:
cd reforestation-CNN

## Install packages

In [ ]:
!pip install -e .
!pip install retry
!pip install geemap
!pip install rasterio

## Import files

In [ ]:
import ee
from src.data.tools.download_gee import GetImageChips
from src.data.tools.rename_image import rename_chips
from src.data.tools.extract_points import retrieve_points
from src.data.tools.crop_image import crop_ref_img
from src.data.tools.crop_image import crop_other_img
from src.data.tools.run_model import segment_images, rgb_predictions, save_geotiff
from src.data.tools import vis
import geemap

In [ ]:
cd notebooks

## Authenticate and Initialize GEE to use the high-volume endpoint

- [high-volume endpoint](https://developers.google.com/earth-engine/cloud/highvolume)

In [ ]:
# Uncoment the next line if you need to authinticate this device
#ee.Authenticate()

ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

## Selection of Regions of Interest

The following notebook cell was created to facilitate the selection of regions for segmentation in the models. You can interact with the map provided by clicking on the highlighted areas, which are represented by an RGB composition of the Atlantic Rainforest boundaries. By clicking on a desired point on the map, a square region with a side length of 2 km will be generated and made available for download to be applied in the models. Please note that although the latitude and longitude coordinates will be recorded in the bottom right corner of the box, no markers will be displayed on the map interface.

In [ ]:
Map = geemap.Map()

basemap =  ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas")\
           .filterDate('2020-06-01','2020-08-31')\
           .select(['R','G','B'])\
           .first()

atlantic = ee.FeatureCollection('projects/ee-fnincao/assets/atlantic_rainfores')

            
vis = {
    'min':0,
    'max':3000,
    'bands':['R', 'G', 'B'],
    'gamma': 1.2}

Map.addLayer(basemap.clip(atlantic), vis, 'Planet RGB')

Map.centerObject(atlantic, 5)

points = retrieve_points(Map)

Map

In [ ]:
#Transform the selected points into a ee.FeatureCollection
geepoints = ee.FeatureCollection([ee.Geometry.Point(lat, lon) for lon, lat in points])

geepoints

## Remote Sensing Imagery pre-processing

The upcoming notebook cells are dedicated to the preprocessing of remote sensing imagery, which is a crucial step in preparing the data for application in the calibrated models. The generated images are yearly composites derived from various sensors.

### Planet Image 2020 (RED / GREEN / BLUE / NEAR-INFRARED) (5m)

In [ ]:
# Create a composite from Planet basemap from 2020
planet_image =  ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas")\
            .filterDate('2020-01-01','2020-12-31')\
            .median()\
            .select('R','G','B','N')

### Planet temporal NDVI image (2016 / 2018 / 2020) (5M)

In [ ]:
def addNDVI(image):
    ndvi = image.normalizedDifference(['N', 'R']).rename('NDVI')
    return image.addBands(ndvi);


# Create a composite from Planet red bands from 2016-2020
planet_ndvi_2016 =  ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas")\
                     .filterDate('2016-01-01', '2016-12-31')\
                     .select(['N','R'])\
                     .map(addNDVI)\
                     .median()\
                     .select('NDVI')

            
planet_ndvi_2018 =  ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas")\
                     .filterDate('2018-01-01', '2018-12-31')\
                     .select(['N','R'])\
                     .map(addNDVI)\
                     .median()\
                     .select('NDVI')

planet_ndvi_2020 = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas")\
                    .filterDate('2020-01-01', '2020-12-31')\
                    .select(['N','R'])\
                    .map(addNDVI)\
                    .median()\
                    .select('NDVI')
           
mosaic_planet_ndvi = planet_ndvi_2016.addBands([planet_ndvi_2018,
                                                planet_ndvi_2020])

### Sentinel-1 Images (Band-C VH) (10m)

In [ ]:
s1_vh_2016 = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterDate('2016-01-01', '2016-12-31')\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))\
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
            .select('VH').median()

s1_vh_2018 = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterDate('2018-01-01', '2018-12-31')\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))\
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
            .select('VH').median()

s1_vh_2020 = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterDate('2020-01-01', '2020-12-31')\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))\
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
            .select('VH').median()

mosaic_s1_vh = s1_vh_2016.addBands([s1_vh_2018,
                                    s1_vh_2020])


### ALOS/PALSAR-2 Images (Band-L HV) (25m)

In [ ]:
palsar_hv_2016 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')\
                  .filter(ee.Filter.date('2016-01-01', '2016-12-31'))\
                  .select('HV')\
                  .first()

palsar_hv_2018 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')\
                  .filter(ee.Filter.date('2018-01-01', '2018-12-31'))\
                  .select('HV')\
                  .first()

palsar_hv_2020 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')\
                   .filter(ee.Filter.date('2020-01-01', '2020-12-31'))\
                   .select('HV')\
                   .first()

mosaic_palsar_hv = palsar_hv_2016.addBands([palsar_hv_2018,
                                            palsar_hv_2020])

### Download the Image Chips

In [ ]:
%%time
GetImageChips(download_image=planet_image,
              out_resolution=5,
              points=geepoints,
              out_dir='../data/gee_data',
              sulfix='_planet')

GetImageChips(download_image=mosaic_planet_ndvi,
              out_resolution=5,
              points=geepoints,
              out_dir='../data/gee_data',
              sulfix='_ndvi')

GetImageChips(download_image=mosaic_s1_vh,
              out_resolution=10,
              points=geepoints,
              out_dir='../data/gee_data',
              sulfix='_s1')

GetImageChips(download_image=mosaic_palsar_hv,
              out_resolution=20,
              points=geepoints,
              out_dir='../data/gee_data',
              sulfix='_palsar')

## Image Cropping

The upcoming cell will perform image cropping to ensure a consistent spatial extent across all images. This process simplifies the data by aligning and standardizing the size of the images, facilitating accurate and reliable analysis using the calibrated models. By establishing a uniform spatial extent, variations in size and alignment are minimized, ensuring consistentcy to subsequent analyses.

In [ ]:
%%time
crop_ref_img(path='../data/gee_data',
             out_dir='../data/croped_data')

crop_other_img(sensor = 'ndvi',
               to_crop_path='../data/gee_data',
               out_dir='../data/croped_data',
               ref_path='../data/croped_data')

crop_other_img(sensor='s1',
               to_crop_path='../data/gee_data',
               out_dir='../data/croped_data',
               ref_path='../data/croped_data')

crop_other_img(sensor='palsar',
               to_crop_path='../data/gee_data',
               out_dir='../data/croped_data',
               ref_path='../data/croped_data')

## Visualize the Image Chips

The following cell functions allow you to visualize the downloaded image chips. You have the option to view individual images or visualize all image chips with the same spatial extent together. Additionally, if you activate the `save_fig` option as `True`, you can save the images in the folder `/data/figures` for future reference.

In [ ]:
from src.data.tools import vis

# Visualize planet chips
#vis.planet_image(tile_number=1, save_fig=True)

# Visualize ndvi chips
#vis.ndvi_image(tile_number=1, save_fig=True)

# Visualize s1 chips
#vis.s1_image(tile_number=1, save_fig=True)

# Visualize Palsar chips
#vis.palsar_image(tile_number=1, save_fig=True)

# Visualize all chips
vis.all_images(tile_number=1, save_fig=True)

## Applying Calibrated Models to Downloaded Data

In the following cells, the calibrated models are applied to the downloaded data. Three different configurations based on the original U-NET architecture are proposed: U-NET RGBN, U-NET NDVI, and U-NET Fusion. Each configuration differs in the Satellite Remote Sensing (SRS) layers used as input data.

### U-NET RGBN
The U-NET RGBN configuration utilizes the 4-band Planet image (Red, Green, Blue, Near-Infrared) 2020 yearly composite as input.

### U-NET NDVI
The U-NET NDVI configuration employs the Planet NDVI temporal 3-bands.

### U-NET Fusion
The U-NET Fusion configuration introduces a novel modification to the U-NET structure, allowing for the inclusion of different SRS layers with varying spatial resolutions. In this configuration, the top level inputs the Planet NDVI image, which is then reduced in spatial resolution by half using the Max Pooling operation. At the second level, the Sentinel-1 image is concatenated, and the same procedure is repeated with the ALOS/PALSAR-2 image at the third level.

The calibrated models were trained using two datasets: Region of Interest 1 (ROI 1), which comprised the whole dataset, and ROI 2, which focused on a subset of the dataset with higher-quality information polygons. The accuracy metrics for each ROI are provided below. To read more insights about the results, please refer to the accompanying report available at the GitHub repository.

### Accuracy Metrics - Region of Interest 1 (ROI 1)
The accuracy metrics for the experiment in ROI 1 are presented in Table below:

| Model        | Overall Accuracy | Dice Score | Precision | Recall |
|--------------|-----------------|------------|-----------|--------|
| U-NET RGBN   | 0.97            | 0.23       | 0.48      | 0.25   |
| U-NET NDVI   | 0.97            | 0.16       | 0.52      | 0.16   |
| U-NET Fusion | 0.97            | 0.26       | 0.51      | 0.31   |

### Accuracy Metrics - Region of Interest 2 (ROI 2)
The accuracy metrics for the experiment in ROI 2 are presented in Table below:

| Model        | Overall Accuracy | Dice Score | Precision | Recall |
|--------------|-----------------|------------|-----------|--------|
| U-NET RGBN   | 0.96            | 0.40       | 0.52      | 0.39   |
| U-NET NDVI   | 0.95            | 0.37       | 0.47      | 0.38   |
| U-NET Fusion | 0.96            | 0.36       | 0.54      | 0.35   |


#### Apply the models Calibrated at ROI 1

In [ ]:
%%time
# Apply the U-NET RGBN calibrated in ROI 1
rgbn_1_preds = segment_images(model_name='rgbn', roi=1)

# Apply the U-NET NDVI calibrated in ROI 1
ndvi_1_preds = segment_images(model_name='ndvi', roi=1)

# Apply the U-NET FUSION calibrated in ROI 1
fusion_1_preds = segment_images(model_name='fusion', roi=1)

#### Apply the models Calibrated at ROI 2

In [ ]:
%%time
# Apply the U-NET RGBN calibrated in ROI 1
rgbn_2_preds = segment_images(model_name='rgbn', roi=2)

# Apply the U-NET NDVI calibrated in ROI 1
ndvi_2_preds = segment_images(model_name='ndvi', roi=2)

# Apply the U-NET FUSION calibrated in ROI 1
fusion_2_preds = segment_images(model_name='fusion', roi=2)

#### Save the prediction in Geotiff format

This step involves saving the predictions in GeoTIFF format at the specified folder `/data/predictions`. The GeoTIFF format allows for the inclusion of georeferencing information, enabling spatial referencing and compatibility with GIS software. By saving the predictions in this format, they can be easily shared and further analyzed or visualized using various geospatial tools and applications.

In [ ]:
#Save predictions from U-NET RGBN calibrated in ROI 1
save_geotiff(pred=rgnb_1_preds, roi=1, model='rgnb')

#Save predictions from U-NET RGBN calibrated in ROI 1
save_geotiff(pred=ndvi_1_preds, roi=1, model='ndvi')

#Save predictions from U-NET RGBN calibrated in ROI 1
save_geotiff(pred=fusion_1_preds, roi=1, model='fusion')

#Save predictions from U-NET RGBN calibrated in ROI 1
save_geotiff(pred=rgbn_2_preds, roi=2, model='rgbn')

#Save predictions from U-NET RGBN calibrated in ROI 1
save_geotiff(pred=ndvi_2_preds, roi=2, model='ndvi')

#Save predictions from U-NET RGBN calibrated in ROI 1
save_geotiff(pred=fusion_2_preds, roi=2, model='fusion')

#### Save Predictions with RGB Background as PNG

The predictions will be saved in PNG format at the folder `/data/predictions`. Saving the predictions in this format allows for easy sharing, analysis, and visualization of the generated results.

In [ ]:
#Save predictions from models calibrated in ROI 1
rgb_predictions(preds_fusion=fusion_1_preds,
                preds_ndvi=ndvi_1_preds,
                preds_rgbn=rgbn_1_preds,
                roi=1)

#Save predictions from models calibrated in ROI 1
rgb_predictions(preds_fusion=fusion_2_preds,
                preds_ndvi=ndvi_2_preds,
                preds_rgbn=rgbn_2_preds,
                roi=2)